# Trénink s destilací nad datasetem DBpedia s modelem BiLSTM
V tomto notebooku je trénován BiLSTM mdoel s uzamčenými embeddingy nad původním i zmenšeným datasetem DBpedia, jako učitelský model je využíván finetunued BERT nad stejným datasetem. V tomto případě není experimentováno s augmentací, a to vzhledem k velikosti datasetu a výborným výsledkům modelů i během normální tréninku. Namísto toho je proveden experiment se zmenšením datasetu (využitím pouze 10 %).

V tomto případě nejsou k dispozici výstupy z prohledávání hyperparametrů, a to z důvodu velikosti datasetu a doby tréninku s ním spojené. Hyperparametry jsou odvozeny z ostatních notebooků s tímto modelem, nicméně nejsou specificky laděné.

Pro úplnost jsou v závěru notebooku spočteny velikost modelu a rychlost inference, avšak využitelnost výstupů nad tímto datasetem není příliš veliká.

## Import knihoven a základní nastavení

In [16]:
from datasets import concatenate_datasets, load_from_disk
from transformers import BasicTokenizer, Trainer, EarlyStoppingCallback
from torch.utils.data import DataLoader
import kagglehub
import torch
import base
import os
import copy

Načtení embeddingů.

Načtení datasetu a jeho základní předzpracování (tokenizace, vytvoření slovníků všech tokenů, vytvoření indexu pro GloVe embeddingy).

In [2]:
my_glove = kagglehub.dataset_download("thanakomsn/glove6b300dtxt")
print(my_glove)

/home/jovyan/.cache/kagglehub/datasets/thanakomsn/glove6b300dtxt/versions/1


In [3]:
GLOVE_FILE = f"{my_glove}/glove.6B.300d.txt"
DATASET = "dbpedia"

In [4]:
train_data = load_from_disk(f"~/data/{DATASET}/train-logits")
eval_data = load_from_disk(f"~/data/{DATASET}/eval-logits")
test_data = load_from_disk(f"~/data/{DATASET}/test-logits")

all_train_data = load_from_disk(f"~/data/{DATASET}/train-logits-augmented")

all_data = concatenate_datasets([load_from_disk(file) for file in [f"~/data/{DATASET}/eval-logits", f"~/data/{DATASET}/test-logits", f"~/data/{DATASET}/train-logits-augmented"]])
tokenizer = BasicTokenizer(do_lower_case=True)

Ověření dostupnosti GPU.

In [5]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 2g.20gb


Tokenizace.

In [6]:
train_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), train_data))
eval_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), eval_data))
test_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), test_data))

all_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), all_data))

Získání všech unikátních tokenů v datasetu.

In [7]:
vocab = base.get_vocab(all_data_tokens)

Přiřazení indexu jednotlivým tokenům.

In [8]:
word_index = dict(zip(vocab, range(len(vocab))))

Získání indexů z GloVe embeddingů.

In [9]:
embeddings_index = base.get_embeddings_indeces(GLOVE_FILE)

Found 400000 word vectors.


Definice velikosti slovníku a velikosti embedding dimenze. 

In [10]:
print(len(vocab))
num_tokens = len(vocab) + 2
embedding_dim = 300

691158


Vytvoření vazby mezi tokeny (jejich indexy) a embeddingy. Spousta tokenů nebyla nalezena, vzhledem k velikosti slovníku nad datasetem je to pochopitelné.


In [11]:
embedding_matrix = base.get_embedding_matrix(num_tokens, embedding_dim, word_index, embeddings_index)

Converted 212978 words (478180) misses


Přiřazení indexu tokenům v každé části datasetu.

In [12]:
train_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),train_data_tokens))
eval_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),eval_data_tokens))
test_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),test_data_tokens))

Zarovnání délky všech záznamů.

In [13]:
train_padded_data = list(map(lambda x: base.padd(x,300), train_data_index))
eval_padded_data = list(map(lambda x: base.padd(x,300), eval_data_index))
test_padded_data = list(map(lambda x: base.padd(x,300), test_data_index))

Přidání ID tokenů do každé části datasetu.

In [14]:
train_data = train_data.add_column("input_ids", train_padded_data)
eval_data = eval_data.add_column("input_ids", eval_padded_data)
test_data = test_data.add_column("input_ids", test_padded_data)

Příprava dataloaderů pro finální ověření rychlosti inference. Testování probíhá pouze nad jedním záznamem z trénovací části.

In [17]:
train_data_gpu = copy.deepcopy(train_data)
train_data_gpu.set_format(type="torch", columns=["input_ids"], device="cuda")
gpu_data_loader = DataLoader(train_data_gpu, batch_size=1, shuffle=False)

train_data_cpu = copy.deepcopy(train_data)
train_data_cpu.set_format(type="torch", columns=["input_ids"], device="cpu")
cpu_data_loader = DataLoader(train_data_cpu, batch_size=1, shuffle=False)

## Normální trénink s původním datasetem

Získání modelu s definovanou embedding vrstvou.

In [18]:
model = base.BiLSTMClassifier(embedding_matrix=embedding_matrix, embedding_dim=embedding_dim, fc_dim=400, hidden_dim=300, output_dim=14)

In [19]:
print(model)

BiLSTMClassifier(
  (embedding): Embedding(691160, 300)
  (lstm): LSTM(300, 300, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=600, out_features=400, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=400, out_features=14, bias=True)
)


Konfigurace tréninku.

In [20]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-base", logging_dir=f"~/logs/{DATASET}/bilstm-base", lr=.005, epochs=5, batch_size=128)

In [21]:
base.reset_seed()

Konfigurace trenéra s trpělivostí 2 epoch. 

In [22]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 2)]
)


Spuštění tréninku, výstupy nad validační částí datasetu.

In [23]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.083200,0.052712,0.986304,0.986310,0.986304,0.986295
2,0.044000,0.047923,0.987491,0.987504,0.987491,0.987489
3,0.029000,0.050115,0.988232,0.988243,0.988232,0.988232
4,0.017000,0.054377,0.988741,0.988787,0.988741,0.988754
5,0.007000,0.066137,0.989116,0.989130,0.989116,0.989122


TrainOutput(global_step=17500, training_loss=0.03604893319266183, metrics={'train_runtime': 699.0689, 'train_samples_per_second': 3204.262, 'train_steps_per_second': 25.033, 'total_flos': 0.0, 'train_loss': 0.03604893319266183, 'epoch': 5.0})

Přepnutí modelu do evaluačního režimu.


In [24]:
model.eval()

BiLSTMClassifier(
  (embedding): Embedding(691160, 300)
  (lstm): LSTM(300, 300, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=600, out_features=400, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=400, out_features=14, bias=True)
)


Otestování modelu nad testovací částí datasetu.

In [25]:
trainer.evaluate(test_data)

{'eval_loss': 0.06466265022754669,
 'eval_accuracy': 0.9892285714285715,
 'eval_precision': 0.98923452190338,
 'eval_recall': 0.9892285714285712,
 'eval_f1': 0.9892306968473393,
 'eval_runtime': 13.2794,
 'eval_samples_per_second': 5271.329,
 'eval_steps_per_second': 41.192,
 'epoch': 5.0}

Uložení modelu.


In [28]:
torch.save(model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/bilstm-base.pth")

## Trénink s destilací s původním datasetem

Získání studentského modelu s definovanou embedding vrstvou.

In [29]:
student_model = base.BiLSTMClassifier(embedding_matrix=embedding_matrix, embedding_dim=embedding_dim, fc_dim=400, hidden_dim=300, output_dim=14)

Konfigurace tréninku.

In [30]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-distill", remove_unused_columns=False, logging_dir=f"~/logs/{DATASET}/bilstm-distill", lr=.005,  epochs=5, batch_size=128, lambda_param=.6, temp=2.5)

In [31]:
base.reset_seed()

Konfigurace destilačního trenéra s trpělivostí 2 epoch. 

In [32]:
trainer = base.DistilTrainer(
    student_model=student_model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 2)]
)

Spuštění tréninku s destilací, výstupy nad validační částí datasetu.

In [33]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.220000,0.096162,0.988179,0.988204,0.988179,0.988178
2,0.099200,0.079958,0.988848,0.988863,0.988848,0.988848
3,0.076300,0.073304,0.989000,0.989010,0.989000,0.989002
4,0.060600,0.065942,0.989107,0.989125,0.989107,0.989109
5,0.048800,0.060589,0.989429,0.989433,0.989429,0.989427


TrainOutput(global_step=17500, training_loss=0.10097243303571428, metrics={'train_runtime': 701.816, 'train_samples_per_second': 3191.72, 'train_steps_per_second': 24.935, 'total_flos': 0.0, 'train_loss': 0.10097243303571428, 'epoch': 5.0})

Přepnutí studenta do evaluačního režimu.

In [34]:
student_model.eval()

BiLSTMClassifier(
  (embedding): Embedding(691160, 300)
  (lstm): LSTM(300, 300, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=600, out_features=400, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=400, out_features=14, bias=True)
)


Otestování modelu nad testovací částí datasetu.

In [35]:
trainer.evaluate(test_data)

{'eval_loss': 0.060866113752126694,
 'eval_accuracy': 0.9890714285714286,
 'eval_precision': 0.9890796152408965,
 'eval_recall': 0.9890714285714285,
 'eval_f1': 0.9890706697046762,
 'eval_runtime': 12.4786,
 'eval_samples_per_second': 5609.601,
 'eval_steps_per_second': 43.835,
 'epoch': 5.0}

Uložení studentského modelu.

In [36]:
torch.save(student_model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/bilstm-distill.pth")

## Normální trénink se zmenšeným datasetem
Zmenšení datasetu stratifikovaným rozdělením na 10 % své původní velikosti.

In [37]:
data = train_data.train_test_split(test_size=0.1, seed=42, stratify_by_column="labels")
train_data = data["test"]

Získání modelu s definovanou embedding vrstvou.

In [38]:
model = base.BiLSTMClassifier(embedding_matrix=embedding_matrix, embedding_dim=embedding_dim, fc_dim=400, hidden_dim=300, output_dim=14)

Konfigurace tréninku.

In [39]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-base-small", logging_dir=f"~/logs/{DATASET}/bilstm-base-small", lr=.005,  epochs=5, batch_size=128)

In [40]:
base.reset_seed()

Konfigurace trenéra s trpělivostí 2 epoch. 

In [41]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 2)]
)


Spuštění tréninku, výstupy nad validační částí datasetu.

In [42]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.311100,0.094965,0.973562,0.973819,0.973562,0.973536
2,0.051700,0.064603,0.983080,0.983122,0.983080,0.983085
3,0.024700,0.071226,0.984062,0.984041,0.984062,0.984041
4,0.010600,0.077239,0.984580,0.984588,0.984580,0.984578
5,0.004300,0.081312,0.985339,0.985339,0.985339,0.985338


TrainOutput(global_step=1750, training_loss=0.08046922445297242, metrics={'train_runtime': 175.4271, 'train_samples_per_second': 1276.884, 'train_steps_per_second': 9.976, 'total_flos': 0.0, 'train_loss': 0.08046922445297242, 'epoch': 5.0})

Přepnutí modelu do evaluačního režimu.


In [43]:
model.eval()

BiLSTMClassifier(
  (embedding): Embedding(691160, 300)
  (lstm): LSTM(300, 300, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=600, out_features=400, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=400, out_features=14, bias=True)
)


Otestování modelu nad testovací částí datasetu.

In [44]:
trainer.evaluate(test_data)

{'eval_loss': 0.0751807913184166,
 'eval_accuracy': 0.9853142857142857,
 'eval_precision': 0.9853046101145619,
 'eval_recall': 0.9853142857142857,
 'eval_f1': 0.9853075627119853,
 'eval_runtime': 12.1207,
 'eval_samples_per_second': 5775.255,
 'eval_steps_per_second': 45.129,
 'epoch': 5.0}

Uložení modelu.


In [45]:
torch.save(model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/bilstm-base-small.pth")

## Trénink s destilací se zmenšeným datasetem
Získání studentského modelu s definovanou embedding vrstvou.

In [46]:
student_model = base.BiLSTMClassifier(embedding_matrix=embedding_matrix, embedding_dim=embedding_dim, fc_dim=400, hidden_dim=300, output_dim=14)

Konfigurace tréninku.

In [47]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-distill-small", remove_unused_columns=False, logging_dir=f"~/logs/{DATASET}/bilstm-distill-small", lr=.004,  epochs=5, batch_size=128, lambda_param=.8, temp=2.5)

In [48]:
base.reset_seed()

Konfigurace destilačního trenéra s trpělivostí 2 epoch. 

In [49]:
trainer = base.DistilTrainer(
    student_model=student_model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 2)]
)

Spuštění tréninku s destilací, výstupy nad validační částí datasetu.

In [50]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.166000,0.217314,0.979652,0.979730,0.979652,0.979616
2,0.157000,0.144653,0.984589,0.984646,0.984589,0.984597
3,0.093500,0.127575,0.985241,0.985242,0.985241,0.985225
4,0.066400,0.113740,0.985714,0.985694,0.985714,0.985697
5,0.053100,0.108999,0.986250,0.986243,0.986250,0.986243


TrainOutput(global_step=1750, training_loss=0.3072006389072963, metrics={'train_runtime': 175.4568, 'train_samples_per_second': 1276.667, 'train_steps_per_second': 9.974, 'total_flos': 0.0, 'train_loss': 0.3072006389072963, 'epoch': 5.0})

Přepnutí studenta do evaluačního režimu.

In [51]:
student_model.eval()

BiLSTMClassifier(
  (embedding): Embedding(691160, 300)
  (lstm): LSTM(300, 300, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=600, out_features=400, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=400, out_features=14, bias=True)
)

Otestování studenta nad testovací částí datasetu.

In [52]:
trainer.evaluate(test_data)

{'eval_loss': 0.1069771945476532,
 'eval_accuracy': 0.986,
 'eval_precision': 0.9859994912121122,
 'eval_recall': 0.9860000000000001,
 'eval_f1': 0.9859939452399261,
 'eval_runtime': 12.2817,
 'eval_samples_per_second': 5699.533,
 'eval_steps_per_second': 44.538,
 'epoch': 5.0}

Uložení studentského modelu.

In [53]:
torch.save(model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/bilstm-distill-small.pth")

Získání počtu trénovatelných parametrů v modelu. 

In [54]:
base.count_parameters(student_model)

model size: 797.420MB.
Total Trainable Params: 1690814.


,Modules,Parameters
0,lstm.weight_ih_l0,360000
1,lstm.weight_hh_l0,360000
2,lstm.bias_ih_l0,1200
3,lstm.bias_hh_l0,1200
4,lstm.weight_ih_l0_reverse,360000
5,lstm.weight_hh_l0_reverse,360000
6,lstm.bias_ih_l0_reverse,1200
7,lstm.bias_hh_l0_reverse,1200
8,fc1.weight,240000
9,fc1.bias,400


Změření rychlosti inference při použití CPU, 1000 pokusů s jedním záznamem.

In [64]:
cpu_benchmark = base.BenchMarkRunner(student_model, cpu_data_loader, "cpu", 1000)
print(cpu_benchmark.run_benchmark())

self.infer_speed_comp()
  19.37 ms
  1 measurement, 1000 runs , 4 threads


Změření rychlosti inference při použití GPU, 1000 pokusů s jedním záznamem.

In [65]:
gpu_benchmark = base.BenchMarkRunner(student_model, gpu_data_loader, "cuda", 1000)
print(gpu_benchmark.run_benchmark())

self.infer_speed_comp()
  5.45 ms
  1 measurement, 1000 runs , 4 threads
